In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Preprocessing of Data

In [7]:
file_path='X:\hemlo\car_price_prediction.csv'
data=pd.read_csv(file_path)
columns_irrelevant=['ID','Model','Levy','Wheel','Color','Leather interior','Engine volume']
data_filtered = data.drop(columns=columns_irrelevant)
columns_to_encode = ['Gear box type', 'Fuel type', 'Manufacturer', 'Category', 'Drive wheels','Doors']
data_encoded = pd.get_dummies(data_filtered, columns=columns_to_encode)
data_encoded['Mileage']=data_encoded['Mileage'].str.replace(' km', '')
data_encoded['Mileage'] = pd.to_numeric(data_encoded['Mileage'])
y=data_encoded['Price']
x=data_encoded.drop(columns=['Price'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)